In [1]:
# Huggett Model adapted from George Hall code, http://people.brandeis.edu/~ghall/econ303/
# tested in Julia 1.5.0
# this code is part of chapter 6, "Heterogeneous Agents Models" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com
using LinearAlgebra;
using SparseArrays;
using Distributions;
using Plots;

#  set parameter values
sigma  = 1.50;            # risk aversion              
beta   = 0.98;            # subjective discount factor 
prob   = [ .8 .2; .5 .5]; # prob(i,j) = probability (s(t+1)=sj | s(t) = si)
theta  = 0.05;            # non-interest income if unemployed
wage   = 1.00;            # non-interest income if employed
Rstart = 1.021;           # initial gross interest rate
F      = -2.0;            # borrowing constraint parameter 
g      = 0.60;            # relaxation parameter

#initialize variables
A      = 1.0;
Aold   = 1.0;
Anew   = 1.0;
meanA  = 1.0;

#form asset grid
maxast = 8;               # maximum value of asset grid   
minast = -5;              # minimum value of asset grid
incast = 0.5;             # size of asset grid increments
nasset = trunc(Int,((maxast-minast)/incast+1)); # number of grid points
assetp = 1.0;

#global variables
decis   = zeros(nasset,2);
tdecis  = zeros(nasset,2);
lambda  = zeros(nasset,2);

#   loop to find R such that sum(lambda*A) = 0 
liter   = 1;
maxiter = 50;
toler   = 0.0001;
step    = 0.05;
R = Rstart;
flag = 1;


In [2]:
println("ITERATING ON R");
println("");
println("Iter       R                                   A                                   newstep");
while  (flag != 0) && (liter <= maxiter);
 
#tabulate the utility function such that for zero or negative
#consumption utility remains a large negative number so that
#such values will never be chosen as utility maximizing      
 
   util1=-10000*ones(nasset,nasset);  # utility when employed     
   util2=-10000*ones(nasset,nasset);  # utility when unemployed   

for i=1:nasset
         asset=(i-1)*incast + minast;
         for j=1:nasset
               assetp = (j-1)*incast + minast;
               cons = wage + R*asset - assetp;
               if assetp >= F && cons > 0;
                  util1[j,i]=(cons)^(1-sigma)/(1-sigma);
               end;
         end
         for j=1:nasset
               assetp = (j-1)*incast + minast;
               cons = theta*wage + R*asset - assetp;
               if assetp>= F && cons > 0;
                  util2[j,i]=(cons)^(1-sigma)/(1-sigma);
               end;
         end;
   end;

#  initialize some variables

   v       = zeros(nasset,2);
   tdecis1 = zeros(nasset,2);
   tdecis2 = zeros(nasset,2);    

 
   test1   = 10;
   test2   = 10;
   rs,cs   = size(util1);
   r1=zeros(nasset,nasset);
   r2=zeros(nasset,nasset);

#  iterate on Bellman's equation and get the decision 
#  rules and the value function at the optimum        
 
 while (test1 != 0) || (test2 > .1);
       for i=1:cs;
           r1[:,i]=util1[:,i]+beta*(prob[1,1]*v[:,1]+ prob[1,2]*v[:,2]);
           r2[:,i]=util2[:,i]+beta*(prob[2,1]*v[:,1]+ prob[2,2]*v[:,2]);
       end;

        
       (tv1,tdecis1)=findmax(r1,dims=1)
       (tv2,tdecis2)=findmax(r2,dims=1);

       for i=1:cs;
       tdecis[i,1]=tdecis1[i][1]
       tdecis[i,2]=tdecis2[i][1]
       end;
       
       tv=[tv1' tv2'];

       test1=maximum((tdecis-decis));
       test2=maximum(abs.(tv-v));
       copy!(v, tv);
       copy!(decis, tdecis); 
   end;
 decis=(decis.-1).*incast.+minast
#   form transition matrix
#   trans is the transition matrix from state at t (row)
#   to the state at t+1 (column) 
 
   g2=spzeros(cs,cs);
   g1=spzeros(cs,cs);
   for i=1:cs
       g1[i,tdecis1[i][1]]=1;
       g2[i,tdecis2[i][1]]=1;
   end
   trans=[ prob[1,1]*g1 prob[1,2]*g1; prob[2,1]*g2 prob[2,2]*g2];
   trans=trans';
   probst = (1/(2*nasset))*ones(2*nasset,1);
   test = 1;
   while test > 10.0^(-8);
       probst1 = trans*probst;
       test = maximum(abs.(probst1-probst));
       copy!(probst, probst1);
   end; 

#   vectorize the decision rule to be conformable with probst
#   calculate new aggregate asset meanA 
aa=vec(decis);
meanA=(probst'*aa)[1];
   
#  calculate measure over (k,s) pairs
#  lambda has same dimensions as decis
lambda=reshape(probst, cs,2)


#   calculate stationary distribution of k
lambda=reshape(probst, cs,2)
probk=sum(lambda',dims=1);     # stationary distribution of capital - sum by each column
probk=probk'

    
if liter == 1;
    A=copy(meanA);; 
if meanA > 0.0;
     step=copy(-step);
end;
end;

Aold = copy(A);
Anew = copy(meanA);

if sign(Aold) != sign(Anew)
     step = copy(-.5*step);
end;
println(liter,"       ",R,"                                  ",meanA,"                             ",step);
if abs.(step) >= toler;
    R=copy(R+step);
else;
      flag = 0;
end;
   A=copy(Anew);
   liter = liter+1;
    
end;

ITERATING ON R

Iter       R                                   A                                   newstep
1       1.021                                  7.381503742789281                             -0.05
2       0.9709999999999999                                  -0.42868278522192527                             0.025
3       0.9959999999999999                                  0.4769046747885715                             -0.0125
4       0.9834999999999999                                  0.01692921100922419                             -0.0125
5       0.971                                  -0.42868278522192527                             0.00625
6       0.97725                                  -0.4286827763313216                             0.00625
7       0.9834999999999999                                  0.01692921100922419                             -0.003125
8       0.9803749999999999                                  0.01692921967959421                             -0.003125
9  

In [3]:
#display solution
#   calculate consumption and expected utility
grid = collect(minast:incast:maxast) ;  
congood = wage*(ones(nasset,1)) + R*grid - grid[floor.(Int, tdecis[:,1])];
conbad  = theta*wage*(ones(nasset,1)) + R*grid - grid[floor.(Int, tdecis[:,2])];
consum  = [congood conbad];
cons2   = [congood.^2 conbad.^2];
meancon = sum(diag(lambda'*consum));
meancon2  = sum(diag(lambda'*cons2));
varcon = ( meancon2 - meancon^2 );
UTILITY = ((complex(consum)).^(1-sigma))./(1-sigma);
UCEU2 = sum(diag(lambda'*UTILITY))

-2.500259434247914 + 0.0im

In [4]:
#print out results
println("PARAMETER VALUES");
println("");
println("sigma      beta      F      theta"); 
println(sigma,"       ",beta,"     ",F,"        ", theta);
println(""); 
println("EQUILIBRIUM RESULTS");
println("");
println("R            A           UCEU    meancon       varcon");
println(round(R,digits=4),"       ",round(meanA,digits=4),"    ",real(round(UCEU2,digits=4)),"    ",round(meancon,digits=4),"     ",round(varcon,digits=4));

PARAMETER VALUES

sigma      beta      F      theta
1.5       0.98     -2.0        0.05

EQUILIBRIUM RESULTS

R            A           UCEU    meancon       varcon
0.9788       -0.4287    -2.5003    0.7377     0.0634
